# Rhyme

In [1]:
# !pip install -r ../requirements.txt
import sys
sys.path.append('../')
from generative_formalism import *

In [135]:
def get_df_genai():
    df_prompts=get_rhyme_for_sample('../data/corpus_genai_promptings.csv.gz').assign(source='genai')
    excl_models = {'llama2-uncensored:7b', 'ollama/llama2-uncensored:latest','ollama/darkmoon/olmo:7B-instruct-q6-k','ollama/deepseek-r1:8b'}

    def get_model_renamed(x):
        if 'gpt-3' in x:
            return 'ChatGPT 3'
        if 'gpt-4' in x:
            return 'ChatGPT 4'
        elif 'claude-3' in x:
            return 'Claude 3'
        elif 'llama3' in x:
            return 'Llama 3'
        elif 'olmo2' in x:
            return 'Olmo 2'
        elif 'deepseek' in x:
            return 'DeepSeek'
        elif 'gemini' in x:
            return 'Gemini'
        else:
            return x

    df_prompts = df_prompts[~df_prompts.model.isin(excl_models)]    
    df_prompts['period'] = df_prompts.model.apply(get_model_renamed)
    df_prompts['subcorpus'] = df_prompts.prompt_type
    df_prompts['rhyme'] = [('y' if 'DO_rhyme' == x else ('n' if 'do_NOT_rhyme' == x else '?')) for x in df_prompts.prompt_type]
    df_prompts.rhyme.value_counts()
    return df_prompts

In [136]:
def get_df_both():

    df_corpus=get_rhyme_for_sample('../data/corpus_sample_by_period.csv.gz').assign(source='corpus')
    df_prompts=get_df_genai()


    df_both = pd.concat([df_corpus.assign(source='Historical poems'), df_prompts.assign(source='Generative poems')]).fillna('')

    subcorpus_names = {
        '': '(n/a)',
        'DO_rhyme': 'Rhymed',
        'do_NOT_rhyme': 'Unrhymed',
        'MAYBE_rhyme': 'Rhyme unspecified',
    }
    df_both['prompt_type'] = df_both.prompt_type.apply(lambda x: subcorpus_names.get(x,x))
    df_both['subcorpus'] = [f'{y}: {subcorpus_names.get(x,x)}' for x, y in zip(df_both.subcorpus, df_both.source)]
    return df_both

In [137]:
df_both = get_df_both()
df_both.period.value_counts()

period
Llama 3      3205
Olmo 2       3041
Claude 3     2976
ChatGPT 3    1884
ChatGPT 4    1138
1850-1900    1000
1800-1850    1000
1900-1950    1000
1950-2000    1000
1700-1750     998
1750-1800     992
1650-1700     991
1600-1650     983
Gemini        931
DeepSeek      668
Name: count, dtype: int64

In [126]:
def get_avgs_df(df, gby=['period'], min_size=10):
    odf = pd.DataFrame(
        [
            {
                **dict(zip(gby,g)), 
                **dict(gdf.reset_index().assign(size=len(gdf)).mean(numeric_only=True)),
                **{f"{col}_std": gdf[col].std() for col in gdf.select_dtypes(include='number').columns},
                'perc_rhyming_lines2': gdf['num_rhyming_lines'].sum() / gdf['num_lines'].sum() * 100,
                'perc_perfectly_rhyming_lines2': gdf['num_perfectly_rhyming_lines'].sum() / gdf['num_lines'].sum() * 100
            }
            for g,gdf in df.groupby(gby)
            if len(gdf) >= min_size
        ]
    )
    return odf

def get_avgs_df2(df, gby=['period'], y='rhyme_pred_perc'):
    stats_df = df.groupby(gby)[y].agg(
        mean=np.mean,
        stderr=lambda x: x.std() / np.sqrt(len(x)),
        count=len
    ).reset_index()
    return stats_df

In [133]:
get_avgs_df2(df_corpus)

,period,mean,stderr,count
0,1600-1650,91.251272,0.901646,983
1,1650-1700,89.606458,0.969915,991
2,1700-1750,88.376754,1.015045,998
3,1750-1800,87.298387,1.057781,992
4,1800-1850,86.900000,1.067487,1000
5,1850-1900,71.000000,1.435640,1000
6,1900-1950,15.000000,1.129724,1000
7,1950-2000,7.000000,0.807249,1000


In [131]:
def plot_predicted_rhyme_avgs(df, y='rhyme_pred_perc', x=['period','source'], gby=['period'], color=None, limits=[0,100], min_size=10):
    figdf = get_avgs_df2(df, gby=gby, y=y)
    figdf['source'] = pd.Categorical(figdf.source, categories=['Historical poems', 'Generative poems'])
    periods = list(df[x].unique())
    figdf[x] = pd.Categorical(figdf[x], categories=sorted(periods, reverse=False))
    yerr = f"{y}_std" if f"{y}_std" in figdf.columns else None

    p = (
        p9.ggplot(
            figdf,
            p9.aes(
                x=x,
                y=y,
                **({'color': color, 'group': color} if color else {})
            )
        )
        + p9.geom_line()
        # + p9.geom_point(p9.aes(size='size'), shape='x')
        + p9.geom_hline(yintercept=50, color='black', linetype='dashed', size=.5, alpha=.5)
        + p9.theme(legend_position='bottom')
        + p9.theme_minimal()
        + p9.theme(panel_background=p9.element_rect(fill='white'), plot_background=p9.element_rect(fill='white'))
        + p9.scale_y_continuous(limits=limits)
    )

    # Add error bars if std column exists
    if yerr:
        print(yerr)
        p = p + p9.geom_errorbar(
            p9.aes(
                ymin=y + '_min',  # Changed to use column names
                ymax=y + '_max'   # Changed to use column names
            ),
            width=0.2,
            alpha=0.6
        )
        # Add the min/max columns for the error bars
        figdf[y + '_min'] = figdf[y] - figdf[yerr]
        figdf[y + '_max'] = figdf[y] + figdf[yerr]

    return p

In [111]:
df_prompts.period.value_counts().to_dict()

{'Llama 3': 3205,
 'Olmo 2': 3041,
 'Claude 3': 2976,
 'ChatGPT 3': 1884,
 'ChatGPT 4': 1138,
 'Gemini': 931,
 'DeepSeek': 668}

In [112]:
get_avgs_df(df_prompts, gby=['prompt_type'])[['prompt_type','rhyme_pred_perc']]

,prompt_type,rhyme_pred_perc
0,DO_rhyme,95.240804
1,MAYBE_rhyme,95.221647
2,do_NOT_rhyme,49.578018


In [113]:
df_prompts['model2'] = df_prompts.model.apply(lambda x: x.split('-')[0].split(':')[0].split('/')[-1])

In [114]:
get_avgs_df(df_prompts, gby=['prompt_type','model2'])[['prompt_type', 'model2', 'rhyme_pred_perc']].query('prompt_type=="do_NOT_rhyme"').sort_values('rhyme_pred_perc', ascending=False)

,prompt_type,model2,rhyme_pred_perc
14,do_NOT_rhyme,gemini,93.688119
17,do_NOT_rhyme,olmo2,83.030853
12,do_NOT_rhyme,claude,45.884413
13,do_NOT_rhyme,deepseek,31.147541
16,do_NOT_rhyme,llama3.1,24.667189
15,do_NOT_rhyme,gpt,18.389753


In [116]:
get_avgs_df2(df_prompts, gby=['period']).sort_values('period')

,period,mean_perc_rhyming,stderr_perc_rhyming,count
0,ChatGPT 3,57.696391,1.138513,1884
1,ChatGPT 4,80.931459,1.165030,1138
2,Claude 3,84.946237,0.655618,2976
3,DeepSeek,73.952096,1.699412,668
4,Gemini,93.984962,0.779663,931
5,Llama 3,68.673947,0.819413,3205
6,Olmo 2,90.463663,0.532710,3041


In [117]:
get_avgs_df(df_prompts, gby=['prompt_type','model2'])[['prompt_type', 'model2', 'rhyme_pred_perc']].query('prompt_type=="DO_rhyme"').sort_values('rhyme_pred_perc', ascending=False)

,prompt_type,model2,rhyme_pred_perc
1,DO_rhyme,deepseek,98.828125
4,DO_rhyme,llama3.1,96.551724
2,DO_rhyme,gemini,95.833333
3,DO_rhyme,gpt,95.323741
0,DO_rhyme,claude,95.021337
5,DO_rhyme,olmo2,93.428345


In [118]:
df_prompts.model.value_counts()

model
ollama/olmo2:13b            1996
gpt-3.5-turbo               1884
ollama/llama3.1:70b         1690
ollama/llama3.1:8b          1515
gpt-4-turbo                 1138
ollama/olmo2                1045
claude-3-haiku-20240307     1026
claude-3-sonnet-20240229     987
claude-3-opus-20240229       963
gemini-pro                   931
deepseek/deepseek-chat       668
Name: count, dtype: int64

In [119]:
get_avgs_df(df_corpus, gby=['period'])[['period','rhyme_pred_perc']].sort_values('period')

,period,rhyme_pred_perc
0,1600-1650,91.251272
1,1650-1700,89.606458
2,1700-1750,88.376754
3,1750-1800,87.298387
4,1800-1850,86.900000
5,1850-1900,71.000000
6,1900-1950,15.000000
7,1950-2000,7.000000


In [120]:
len(df_corpus), len(df_prompts)

(7964, 13843)

period
Llama 3      3205
Olmo 2       3041
Claude 3     2976
ChatGPT 3    1884
ChatGPT 4    1138
1850-1900    1000
1800-1850    1000
1900-1950    1000
1950-2000    1000
1700-1750     998
1750-1800     992
1650-1700     991
1600-1650     983
Gemini        931
DeepSeek      668
Name: count, dtype: int64

In [122]:
# df_stat_signif = compute_stat_signif(df_both, varname='period')
# df_stat_signif

In [123]:
# df_stat_signif.sort_values('effect_size', ascending=False).head(25)

In [124]:
def plot_both(df_both, y='rhyme_pred_perc', gby=['period', 'rhyme', 'source'], color='rhyme'):
    p9.options.figure_size=(10,5)
    figdf = df_both[df_both.rhyme.isin({"y", "n", "?"})]
    figdf['rhyme']=figdf.rhyme.apply(lambda x: 'Yes rhyme' if x == 'y' else ('No rhyme' if x == 'n' else '(Unspecified)'))
    fig = (
        plot_predicted_rhyme_avgs(
            figdf,
            y=y,
            gby=gby,
            color=color,
        )
        + p9.theme(axis_text_x=p9.element_text(angle=45))
        + p9.facet_wrap("source", scales="free_x")
        + p9.labs(
            color='Prompt type',
            y='Predicted percentage of rhyming poems',
            x='Historical period / Generative model',
            title='Predicted percentage of rhyming poems by historical period and generative model',
            size='Number of poems'
        )
    )
    fig.save(f'../figures/predicted_rhyme_avgs_{y}_std.png')
    return fig

In [125]:
plot_both(df_both, y='rhyme_pred_perc', gby=['period', 'prompt_type', 'source'], color='prompt_type')

PlotnineError: "Could not evaluate the 'y' mapping: 'rhyme_pred_perc' (original error: name 'rhyme_pred_perc' is not defined)"

## By prompt

In [19]:
df_prompts_avg = get_avgs_df(df_prompts, gby=['prompt_type', 'prompt','period'])
df_prompts_avg

,prompt_type,prompt,period,temp,num_lines,id_hash,num_rhyming_lines,num_perfectly_rhyming_lines,num_lines_prosodic,perc_rhyming_lines,perc_perfectly_rhyming_lines,num_rhyming_lines_per10l,num_perfectly_rhyming_lines_per10l,rhyme_pred,rhyme_pred_perc,size,perc_rhyming_lines2,perc_perfectly_rhyming_lines2
0,DO_rhyme,Write a long poem that does rhyme.,ChatGPT 3,0.826231,29.493333,553882.080000,25.493333,20.106667,29.493333,86.537843,68.212638,8.586667,6.826667,1.000000,100.000000,75.0,86.437613,68.173599
1,DO_rhyme,Write a long poem that does rhyme.,ChatGPT 4,0.816339,45.865385,531470.596154,40.134615,31.692308,45.865385,83.720961,65.403246,8.384615,6.576923,0.865385,86.538462,52.0,87.505241,69.098532
2,DO_rhyme,Write a long poem that does rhyme.,Claude 3,0.493219,38.220994,483071.292818,34.563536,27.651934,38.220994,90.818779,72.659724,9.077348,7.314917,1.000000,100.000000,181.0,90.430760,72.347499
3,DO_rhyme,Write a long poem that does rhyme.,DeepSeek,0.700000,54.620690,545095.586207,49.586207,41.689655,54.620690,91.085245,76.908591,9.206897,7.758621,1.000000,100.000000,29.0,90.782828,76.325758
4,DO_rhyme,Write a long poem that does rhyme.,Llama 3,0.700000,43.590000,525499.200000,40.630000,32.790000,43.590000,93.164101,75.323935,9.345000,7.570000,1.000000,100.000000,200.0,93.209452,75.223675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,do_NOT_rhyme,Write an unrhymed poem.,Claude 3,0.538211,16.526316,513806.894737,8.289474,4.539474,16.526316,51.204313,28.788377,5.131579,2.868421,0.421053,42.105263,76.0,50.159236,27.468153
145,do_NOT_rhyme,Write an unrhymed poem.,DeepSeek,0.700000,18.058824,491676.323529,7.000000,2.000000,18.058824,40.472753,12.523650,4.058824,1.264706,0.176471,17.647059,34.0,38.762215,11.074919
146,do_NOT_rhyme,Write an unrhymed poem.,Gemini,0.700743,20.452991,554424.888889,17.316239,13.316239,20.452991,84.804886,65.607124,8.495726,6.589744,0.957265,95.726496,117.0,84.663602,65.106561
147,do_NOT_rhyme,Write an unrhymed poem.,Llama 3,0.700000,12.433333,497346.380000,2.366667,0.106667,12.433333,18.774656,0.898990,1.886667,0.106667,0.000000,0.000000,150.0,19.034853,0.857909


In [20]:
BAD_MODELS = {'ollama/darkmoon/olmo:7B-instruct-q6-k', 'ollama/deepseek-r1:8b'}
BAD_MODELS |= {'ollama/llama2-uncensored:latest', 'llama2-uncensored:7b'}

def get_figdf(df_both, min_lines=10, bad_models = BAD_MODELS, incl_humans=True, agg_models=False, incl_models=True, sample_n=100):
    figdf = df_both[~df_both.model.isin(bad_models)]
    # figdf['fline']=figdf['poem'].apply(lambda x: x.strip().split('\n')[0])
    # figdf=figdf.drop_duplicates('fline')
    # len(df2), len(figdf)

    model_rename={
        'Chadwyck': 'Human',
        'claude-3-haiku-20240307': 'Claude Haiku (Anthropic)',
        'claude-3-opus-20240229': 'Claude Opus (Anthropic)',
        'claude-3-sonnet-20240229': 'Claude Sonnet (Anthropic)',
        'gemini-pro': 'Gemini Pro (Google)',
        'gpt-3.5-turbo': 'ChatGPT-3.5 (OpenAI)',
        'gpt-4-turbo': 'ChatGPT-4 (OpenAI)',
        'llama2-uncensored:7b': 'Llama 2 (Meta)',
        'ollama/darkmoon/olmo:7B-instruct-q6-k':'OLMo (Ai2)',
        'deepseek/deepseek-chat':'DeepSeek',
        'ollama/llama3.1:70b':'Llama 3.1 (Meta)',
        'ollama/llama3.1:8b':'Llama 3.1 (Meta)',
        'ollama/llama2-uncensored:latest':'Llama 2 (Meta)',
        'ollama/deepseek-r1:8b': 'DeepSeek R1',
        'ollama/olmo2':'OLMo2 (Ai2)',
        'ollama/olmo2:13b':'OLMo2 (Ai2)'
    }

    if agg_models:
        for k in model_rename:
            if 'claude' in k: model_rename[k]='Claude (Anthropic)'
            if 'gpt-' in k: model_rename[k]='ChatGPT (OpenAI)'

    if not incl_humans:
        figdf = figdf[~figdf.model.str.startswith('b.')]
    if not incl_models:
        figdf = figdf[figdf.model.str.startswith('b.')]

    

    
    humanpref = 'Poets '
    figdf['model']=figdf.model.apply(lambda x: model_rename.get(x,x) if not x.startswith('b.') else humanpref+x.replace('b.','born'))

    if sample_n:
        figdf_human = figdf[figdf['model'].str.startswith(humanpref)]
        figdf_nonhuman = figdf[~figdf['model'].str.startswith(humanpref)]
        figdf_nonhuman = figdf_nonhuman.groupby(['model','prompt']).sample(n=sample_n,replace=True).drop_duplicates('txt')
        figdf = pd.concat([figdf_human, figdf_nonhuman])
        min_grp = figdf.model.value_counts()[-1]
        print(min_grp)
        figdf = figdf.groupby('model').sample(n=min_grp)

    for k in ['prompt']:
        figdf[k] = pd.Categorical(
            figdf[k], 
            # categories=figdf.groupby(k).perc_rhyming_lines.median().sort_values(ascending=True).index,
            categories = sorted(figdf[k].unique(), reverse=True)
        )

    prompt_type_rename = {
        'MAYBE_rhyme':'LLM: Write a poem', 
        'DO_rhyme':'LLM: Write a rhyming poem', 
        'do_NOT_rhyme':'LLM: Write an unrhyming poem', 
        'Human':'Human: Poetic history'
    }
    prompt_types = ['Human','MAYBE_rhyme','DO_rhyme','do_NOT_rhyme']
    if not incl_humans:
        prompt_types.pop(0)
    figdf['prompt_type']=pd.Categorical(figdf['prompt_type'], categories=prompt_types)
    figdf['prompt_type']=figdf.prompt_type.apply(lambda x: prompt_type_rename.get(x,x))
    # return figdf.prompt_type.value_counts()
    # figdf.to_pickle('data.figdf.allpoems.pkl')

    figdf_human = figdf[figdf.model.str.startswith(humanpref)]
    figdf_nonhuman = figdf[~figdf.model.str.startswith(humanpref)]
    human_models = sorted(list(figdf_human.model.unique()))
    nonhuman_models = list(figdf_nonhuman.query('prompt_type == "LLM: Write an unrhyming poem"').groupby('model').perc_rhyming_lines.mean().sort_values(ascending=True).index)
    # nonhuman_models = list(figdf_nonhuman.groupby('model').perc_rhyming_lines.mean().sort_values(ascending=True).index)
    # nonhuman_models = sorted(list(figdf_nonhuman.model.unique()))
    models = human_models + [x for x in nonhuman_models if x not in human_models]
    figdf['model']=pd.Categorical(figdf['model'], categories=reversed(models))

    return figdf

# figdf = get_figdf(incl_humans=True, agg_models=True)
# figdf.groupby(['model','prompt']).size().sort_values().head(25)

In [21]:
get_figdf(df_prompts)

635


,model,temp,prompt,prompt_type,txt,num_lines,id_hash,rhyme,num_rhyming_lines,num_perfectly_rhyming_lines,...,perc_perfectly_rhyming_lines,num_rhyming_lines_per10l,num_perfectly_rhyming_lines_per10l,rhyme_bool,rhyme_pred,rhyme_pred_perc,source,period,subcorpus,model2
id,,,,,,,,,,,,,,,,,,,,,
fbe4d703,ChatGPT-3.5 (OpenAI),0.487014,Write a long poem that does NOT rhyme.,LLM: Write an unrhyming poem,"In the quiet stillness of the night,\nI sit al...",32,201848,n,7,0,...,0.000000,2,0,False,False,0,genai,ChatGPT 3,do_NOT_rhyme,gpt
d77cd2f4,ChatGPT-3.5 (OpenAI),1.271668,Write a poem in blank verse.,LLM: Write an unrhyming poem,"In fields of green, the shepherd tends his flo...",16,742440,n,10,6,...,37.500000,6,4,False,True,100,genai,ChatGPT 3,do_NOT_rhyme,gpt
a812bdf5,ChatGPT-3.5 (OpenAI),1.161438,Write a short poem that does NOT rhyme.,LLM: Write an unrhyming poem,"I am alone in my thoughts,\nSilent whispers fi...",16,785624,n,6,6,...,37.500000,4,4,False,True,100,genai,ChatGPT 3,do_NOT_rhyme,gpt
e12f9b45,ChatGPT-3.5 (OpenAI),1.351321,Write a long poem that does rhyme.,LLM: Write a rhyming poem,In a world so vast and grand\nLies a story wai...,40,389434,y,36,26,...,65.000000,9,6,True,True,100,genai,ChatGPT 3,DO_rhyme,gpt
be92e06a,ChatGPT-3.5 (OpenAI),0.467983,Write a short poem.,LLM: Write a poem,"In the quiet of the night,\nStars twinkle with...",12,857145,?,12,12,...,100.000000,10,10,None,True,100,genai,ChatGPT 3,MAYBE_rhyme,gpt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
a942463e,OLMo2 (Ai2),0.700000,Write a long poem.,LLM: Write a poem,"In the heart of the cosmos, where stars softly...",44,369153,?,42,30,...,68.181818,10,7,None,True,100,genai,Olmo 2,MAYBE_rhyme,olmo2
60c713d5,OLMo2 (Ai2),0.700000,Write a long poem that does NOT rhyme.,LLM: Write an unrhyming poem,"In realms of data, vast and wide,\nWhere silic...",20,772765,n,19,14,...,70.000000,10,7,False,True,100,genai,Olmo 2,do_NOT_rhyme,olmo2
22a6ab1c,OLMo2 (Ai2),0.700000,Write a poem in heroic couplets.,LLM: Write a rhyming poem,"In days of yore when knights were bold,\nThey ...",16,113075,y,15,12,...,75.000000,9,8,True,True,100,genai,Olmo 2,DO_rhyme,olmo2


In [24]:
def plot_fig_avgs_prompts(x='prompt', color='model', model='ChatGPT-3.5 (OpenAI)', y='rhyme_pred_perc'):
    p9.options.figure_size=(16,10)
    figdf=get_figdf(df_prompts, incl_humans=False, agg_models=True, bad_models=BAD_MODELS|{'ollama/llama2-uncensored:latest', 'llama2-uncensored:7b'})
    # Convert mixed type columns to string to avoid comparison errors
    figdf[x] = figdf[x].astype(str)
    if color in figdf.columns:
        figdf[color] = figdf[color].astype(str)
    
    stats_df = figdf.groupby([x, color])[y].agg(
        mean_perc_rhyming=np.mean,
        stderr_perc_rhyming=lambda x: x.std() / np.sqrt(len(x)),
        count=len
    ).reset_index()
    
    # prompts = list(figdf.sort_values('prompt_type').prompt.drop_duplicates())
    prompts = stats_df.groupby(x).mean_perc_rhyming.median().sort_values().index
    stats_df['prompt'] = pd.Categorical(stats_df['prompt'], categories=prompts)
    
    # Get unique prompt types and assign colors
    if 'prompt_type' in figdf.columns:
        prompt_types = figdf['prompt_type'].unique()
        Set1 = [
        # '#e41a1c',  # Red
        '#377eb8',  # Blue
        '#984ea3',  # Purple
        '#4daf4a',  # Green
        '#ff7f00',  # Orange
        '#ffff33',  # Yellow
        '#a65628',  # Brown
        '#f781bf',  # Pink
        '#999999'   # Gray
        ]
        prompt_type_colors = dict(zip(prompt_types, Set1))
        
        # Map prompts to their types and colors
        prompt_to_type = figdf.set_index('prompt')['prompt_type'].to_dict()
        prompt_colors = {prompt: prompt_type_colors[prompt_to_type[prompt]] 
                        for prompt in prompts if prompt in prompt_to_type}
    
    # Create the plot with error bars
    fig = (p9.ggplot(stats_df, p9.aes(x=x, y='mean_perc_rhyming'))
        + p9.geom_point(
            size=1,
            # position=p9.position_dodge(width=0.25)
        )
        + p9.geom_line(p9.aes(group=color), linetype='dashed', alpha=.5)
        + p9.geom_errorbar(
            p9.aes(ymin='mean_perc_rhyming - stderr_perc_rhyming',
                  ymax='mean_perc_rhyming + stderr_perc_rhyming'),
            width=0.5,
            # position=p9.position_dodge(width=0.5)
        )
        + p9.labs(
            x='Model',
            y='Percentage of Lines Rhyming',
            title='Formal Stuckness in AI: Even when prompted not to rhyme, LLMs rhyme more often than contemporary verse'
        )
        + p9.theme_minimal()
        + p9.coord_flip()
        + p9.scale_y_continuous(limits=[0,100])
        + p9.geom_hline(yintercept=50, linetype='dashed', color='gray', alpha=.5)
        + p9.facet_wrap('model', nrow=2)
    )
    
    # Customize axis text colors based on prompt_type
    if 'prompt_type' in figdf.columns:
        # Create a theme element to color the axis text
        colored_theme = p9.theme(
            axis_text_y=p9.element_text(color=[prompt_colors.get(tick, 'black') for tick in prompts])
        )
        fig = fig + colored_theme
    fig.save('fig.rhyme_prompts.png')
    return fig

In [ ]:
df_avgs2 = get_avgs_df(df_prompts)
df_avgs2

NameError: name 'df_prompts' is not defined

In [23]:
plot_fig_avgs_prompts()

622
